INSTALL
- FAISS - 
- Sent2Vec

In [1]:
import numpy as np
import faiss   

In [2]:
import os
import time
name = "Faiss-EM-LAW-sent2vec-en-vi-0.999-"+ str(time.strftime("%Y%m%d-%H%M"))

!mkdir $name
path = os.getcwd() + "/" + name
os.chdir(path)

!pwd


os.chdir(path)

/home/chinh/clc_data/Script/Faiss-EM-LAW-sent2vec-en-vi-0.999-20210407-2000


In [3]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/law.tar.gz
!mkdir data_bin
!tar -xvf  'law.tar.gz' -C 'data_bin'

--2021-04-07 20:00:24--  https://github.com/hoangtrungchinh/clc_data/raw/master/law.tar.gz
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/law.tar.gz [following]
--2021-04-07 20:00:25--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/law.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 988814 (966K) [application/octet-stream]
Saving to: ‘law.tar.gz’

law.tar.gz          100%[===================>] 965,64K  3,04MB/s    in 0,3s    

2021-04-07 20:00:26 (3,04 MB/s) - ‘law.tar.gz’ saved [988814/988814]

_law.vi
_law.en


In [4]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
vi_files = sorted(glob.glob("data_bin/*.vi"))
print(en_files)
print(vi_files)

lst_en = []
lst_vi = []

start = time.time()

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line)

for file in vi_files:
  with open(file) as file_in:
    for line in file_in:
      lst_vi.append(line)

print(len(lst_en))
print(len(lst_vi))

['data_bin/_law.en']
['data_bin/_law.vi']
17967
17967


In [5]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, vi_train, vi_test_valid =  train_test_split(lst_en, lst_vi, test_size=0.2, random_state=123)
en_valid, en_test, vi_valid, vi_test = train_test_split(en_test_valid, vi_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(vi_train))
print(len(en_valid))
print(len(vi_valid))
print(len(en_test))
print(len(vi_test))
print(en_train[20])
print(vi_train[20])
print(en_valid[-1])
print(vi_valid[-1])
print(en_test[-1])
print(vi_test[-1])

14373
14373
1797
1797
1797
1797
A person aged from fifteen years to below eighteen years may be an authorized representative, except where the law provides for that the civil transaction must be entered into and performed by a person who has reached eighteen years of age.

Người từ đủ mười lăm tuổi đến chưa đủ mười tám tuổi có thể là người đại diện theo uỷ quyền, trừ trường hợp pháp luật quy định giao dịch dân sự phải do người từ đủ mười tám tuổi trở lên xác lập, thực hiện.

The Law on Bankruptcy is applied to the bankruptcy process of every enterprise and cooperative based in the territory of Vietnam.

Luật Phá sản được áp dụng khi giải quyết phá sản đối với doanh nghiệp, hợp tác xã được thành lập trên lãnh thổ Cộng hòa xã hội chủ nghĩa Việt Nam.

A lawsuit is instituted over an administrative decision or act of a provincial-level People’s Committee or provincial-level People’s Committee chairperson which is related to many subjects;

Khiếu kiện quyết định hành chính, hành vi hành chí

In [6]:
file_variable = [en_train,vi_train,en_valid,vi_valid,en_test,vi_test]
file_names = ["en_train","vi_train","en_valid","vi_valid","en_test","vi_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item)
!ls -al

total 6672
drwxrwxr-x  3 chinh chinh    4096 Thg 4   7 20:00 .
drwxrwxr-x 14 chinh chinh    4096 Thg 4   7 20:00 ..
drwxrwxr-x  2 chinh chinh    4096 Thg 4   7 20:00 data_bin
-rw-rw-r--  1 chinh chinh  257314 Thg 4   7 20:00 en_test
-rw-rw-r--  1 chinh chinh 2096540 Thg 4   7 20:00 en_train
-rw-rw-r--  1 chinh chinh  265528 Thg 4   7 20:00 en_valid
-rw-rw-r--  1 chinh chinh  988814 Thg 4   7 20:00 law.tar.gz
-rw-rw-r--  1 chinh chinh  314903 Thg 4   7 20:00 vi_test
-rw-rw-r--  1 chinh chinh 2560025 Thg 4   7 20:00 vi_train
-rw-rw-r--  1 chinh chinh  325494 Thg 4   7 20:00 vi_valid


In [7]:
start = time.time()
import sent2vec
model = sent2vec.Sent2vecModel()
model.load_model('../../../wiki_unigrams.bin')

print('=== Ending, Total time (second): ', time.time() - start)

=== Ending, Total time (second):  9.010917663574219


In [8]:
srcfile = "en_train"
tarfile = "vi_train"
# lst_srcfile = open(srcfile, "r").readlines()
# lst_tarfile = open(tarfile, "r").readlines()
lst_srcfile = en_train
lst_tarfile = vi_train

vec_lst_srcfile = model.embed_sentences(lst_srcfile)

Test Cosin similarity

In [9]:
import faiss

start = time.time()
vec_lst_srcfile = model.embed_sentences(lst_srcfile)
index = faiss.IndexFlatL2(600)
index.ntotal
faiss.normalize_L2(vec_lst_srcfile)
index.add(vec_lst_srcfile)
k = 6
distance, index = index.search(vec_lst_srcfile, k)

print('=== Ending, Total time (second): ', time.time() - start)
print(len(distance))
print(len(index))

print(index)
print(distance)

=== Ending, Total time (second):  0.720555305480957
14373
14373
[[    0  4092  4120  3880 12276 11292]
 [    1  7317  3514  7883  9575 13622]
 [    2 11962  1745   514   378   583]
 ...
 [14370  6329 12151  6032  5663  9164]
 [14371  5816  8185 13903 13816  4554]
 [14372  8951 11020   127 11507   514]]
[[0.0000000e+00 4.5765066e-01 4.7214556e-01 5.1959085e-01 5.5730844e-01
  5.6726301e-01]
 [0.0000000e+00 5.3958035e-01 5.5549896e-01 6.5495539e-01 6.5672672e-01
  6.7501473e-01]
 [1.1920929e-07 4.8932076e-01 6.9507873e-01 9.9999994e-01 9.9999994e-01
  9.9999994e-01]
 ...
 [0.0000000e+00 6.6308725e-01 7.3182833e-01 8.1565511e-01 8.5339057e-01
  8.5837018e-01]
 [0.0000000e+00 1.7185676e-01 2.0529890e-01 3.2067645e-01 3.9019966e-01
  4.4887304e-01]
 [1.1920929e-07 4.5519710e-01 9.6420062e-01 9.6752667e-01 9.8171270e-01
  1.0000000e+00]]


In [10]:
print(type(vec_lst_srcfile))
print(vec_lst_srcfile.shape)


<class 'numpy.ndarray'>
(14373, 600)


In [11]:
i = 6
j = k-1
print(index[i][j])
print("==================================")
print(distance[i][j])
a = vec_lst_srcfile[i]
# print(a)
b = vec_lst_srcfile[index[i][j]]
# TEST
dist_squared = np.sum(np.square(a - b))
print(dist_squared)

11445
0.8651743
0.8651744


In [12]:
start = time.time()
saperate = " || "

src_label = "S"
emb_label = "E"
thres_arr = [0.8, 0.85, 0.9, 0.95, 0.99]
for i in range(len(thres_arr)):
    threshold = thres_arr[i]
    out_file_write = open("en_train_EM_"+str(thres_arr[i]), 'w')
    out_file_write_score = open("en_train_EM_score_"+str(thres_arr[i]), 'w')
    out_file_write_factor = open("en_train_EM_factor_"+str(thres_arr[i]), 'w')
    num_gth_thres = 0

    for i in range(len(lst_srcfile)):
        score = distance[i][1]
        content = ""
        content_label_arr = []
        if score >= threshold:
            num_gth_thres += 1
            best_simi_index = index[i][1]
            content = lst_srcfile[i].strip() + saperate + lst_tarfile[best_simi_index].strip()

            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label.append(emb_label)
            content_label = content_label + [emb_label] * len(lst_tarfile[best_simi_index].split())

            content_label_arr = " ".join(content_label)
        else:
            content = lst_srcfile[i].strip()
            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label_arr = " ".join(content_label)
 
        out_file_write.writelines(content+ "\n")    
        out_file_write_factor.writelines(content_label_arr + "\n")
        out_file_write_score.writelines(str(score) + "\n")

    print('threshold =', threshold, "EM sentences =", num_gth_thres, "/", len(lst_srcfile), num_gth_thres/len(lst_srcfile), "%" )

    out_file_write.close()
    out_file_write_score.close()
    out_file_write_factor.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)

threshold = 0.8 EM sentences = 2012 / 14373 0.13998469352257706 %
threshold = 0.85 EM sentences = 1418 / 14373 0.09865720448062339 %
threshold = 0.9 EM sentences = 947 / 14373 0.06588742781604397 %
threshold = 0.95 EM sentences = 607 / 14373 0.04223196270785501 %
threshold = 0.99 EM sentences = 431 / 14373 0.029986780769498365 %
=== Ending, Total time (second):  1.168128252029419
